In [1]:
import mysql.connector

In [39]:
class SaveToNewDb:
    def __init__(self, db):
        self.mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database=db
        )
        self.mycursor = self.mydb.cursor(dictionary=True)
    
    def _execute_and_commit(self, sql, values):
        self.mycursor.execute(sql, values)
        self.mydb.commit()
    
    def last_item(self, table):
        sql= f"select * from {table} getLastRecord ORDER BY UID DESC LIMIT 1"
        self.mycursor.execute(sql)
        last_record = self.mycursor.fetchone()
        return last_record['UID']
    
    def add_old_id_column(self, table):
        try:
            self.mycursor.execute(f"ALTER TABLE `{table}` ADD COLUMN `old_id` VARCHAR(100);")
            mydb.commit()
            print(f"column `old_id` added successfully to {table}...")
        except:
            print(f"column `old_id` already exist in {table}!")
    
    def add_status_column(self, table):
        try:
            self.mycursor.execute(f"ALTER TABLE `{table}` ADD COLUMN `status`VARCHAR(100) DEFAULT 'NOT UPDATED';")
            mydb.commit()
            print(f"column `status` added successfully to {table}...")
        except:
            print(f"column `status` already exist in {table}!")
                
                
    def save_accessories(self, parent, title_en, title_fa, short_description_en, short_description_fa, 
                         long_description_en, long_description_fa, page_url):
        sql = """INSERT INTO `accessories`
                 (`parent_UID`, `title_en`, `title_fa`, `short_description_en`, `short_description_fa`, 
                 `long_description_en`, `long_description_fa`, `page_url`) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (parent, title_en, title_fa, short_description_en, short_description_fa, long_description_en,
                  long_description_fa, page_url)
        self._execute_and_commit(sql, values)
        print("Saved in accessories successfully...")
        
        
    def save_attributes(self, parent_id, parent_type, brand_en, brand_fa, key_en, key_fa, value_en, value_fa):
        sql = """INSERT INTO `attributes`(`parent_UID`, `parent_type`, `brand_en`, `brand_fa`, `key_en`, `key_fa`, 
                                          `value_en`, `value_fa`) 
                 VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
        values = (parent_id, parent_type, brand_en, brand_fa, key_en, key_fa, value_en, value_fa)
        self._execute_and_commit(sql, values)
        print("Attributes updated successfully...")


    def save_categories(self, parent_id, title_en, title_fa, short_description_en, short_description_fa, 
                        long_description_en, long_description_fa, page_url, old_id):
        sql = """INSERT INTO `categories`
                 (`parent_UID`, `title_en`, `title_fa`, `short_description_en`, `short_description_fa`,
                 `long_description_en`, `long_description_fa`, `page_url`, `old_id`) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (parent_id, title_en, title_fa, short_description_en, short_description_fa, 
                  long_description_en, long_description_fa, page_url, old_id)
        self._execute_and_commit(sql, values)
        print("Saved in categories successfully...")
        
        updated_id = self.last_item("categories")
        
        # Update parent_id in products
        self.mycursor.execute(f"""UPDATE products SET category_UID='{int(updated_id)}', status='UPDATED' 
                                  WHERE UID='{int(old_id)}' AND status='NOT UPDATED'""")
        self.mydb.commit()
 
        # Update parent_id in images
        self.mycursor.execute(f"""UPDATE images SET parent_UID={int(updated_id)},  status='UPDATED' 
                                  WHERE parent_UID={int(old_id)} AND parent_type='categories' AND  status='NOT UPDATED'""")
        self.mydb.commit()

        # Update parent_id in pdfs
        self.mycursor.execute(f"""UPDATE pdfs SET parent_UID={int(updated_id)},  status='UPDATED' 
                                  WHERE parent_UID={int(old_id)} AND parent_type='categories' AND  status='NOT UPDATED'""")
        self.mydb.commit()

        # Update parent_id in attributes
        self.mycursor.execute(f"""UPDATE attributes SET parent_UID={int(updated_id)},  status='UPDATED'
                                  WHERE parent_UID={int(old_id)} AND parent_type='categories' AND  status='NOT UPDATED'""")
        self.mydb.commit()
        
        # Update parent_id in videos
        self.mycursor.execute(f"""UPDATE videos SET parent_UID={int(updated_id)},  status='UPDATED'
                                  WHERE parent_UID={int(old_id)} AND parent_type='categories' AND  status='NOT UPDATED'""")
        self.mydb.commit()
        
        # Update parent_id in accessories
        self.mycursor.execute(f"""UPDATE videos SET parent_UID={int(updated_id)},  status='UPDATED'
                                  WHERE parent_UID={int(old_id)} AND parent_type='categories' and status='NOT UPDATED'""")
        self.mydb.commit()


    def save_products(self, parent_id, title_en, title_fa, product_UOM, code, barcode, 
                   short_description_en, short_description_fa, long_description_en, long_description_fa,
                   page_url, accessories, old_id):
        sql = """INSERT INTO `products`
                 (`category_UID`, `title_en`, `title_fa`, `product_UOM`, `code`, `barcode`, 
                 `short_description_en`, `short_description_fa`, `long_description_en`, `long_description_fa`,
                 `page_url`, `accessories`) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (parent_id, title_en, title_fa, product_UOM, code, barcode, 
                  short_description_en, short_description_fa, long_description_en, long_description_fa,
                  page_url, accessories)
        self._execute_and_commit(sql, values)
        print("Saved in codes successfully...")
        
        updated_id = self.last_item("products")
        
 
        # Update parent_id in images
        self.mycursor.execute(f"""UPDATE images SET parent_UID={int(updated_id)},  status='UPDATED' 
                                  WHERE parent_UID={int(old_id)} AND parent_type='products' AND  status='NOT UPDATED'""")
        self.mydb.commit()

        # Update parent_id in pdfs
        self.mycursor.execute(f"""UPDATE pdfs SET parent_UID={int(updated_id)},  status='UPDATED' 
                                  WHERE parent_UID={int(old_id)} AND parent_type='products' AND  status='NOT UPDATED'""")
        self.mydb.commit()

        # Update parent_id in attributes
        self.mycursor.execute(f"""UPDATE attributes SET parent_UID={int(updated_id)},  status='UPDATED'
                                  WHERE parent_UID={int(old_id)} AND parent_type='products' AND  status='NOT UPDATED'""")
        self.mydb.commit()
        
        # Update parent_id in videos
        self.mycursor.execute(f"""UPDATE videos SET parent_UID={int(updated_id)},  status='UPDATED'
                                  WHERE parent_UID={int(old_id)} AND parent_type='products' AND  status='NOT UPDATED'""")
        self.mydb.commit()
        
        # Update parent_id in accessories
        self.mycursor.execute(f"""UPDATE videos SET parent_UID={int(updated_id)},  status='UPDATED'
                                  WHERE parent_UID={int(old_id)} AND parent_type='products' and status='NOT UPDATED'""")
        self.mydb.commit()

        
    def save_images(self, parent_id, parent_type, image_URL, 
                    alt_en, alt_fa, description_en, description_fa):
        sql = """INSERT INTO `images`
                 (`parent_UID`, `parent_type`, `image_URL`, 
                 `alt_en`, `alt_fa`, `description_en`, `description_fa`) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s)"""
        values = (parent_id, parent_type, image_URL, 
                  alt_en, alt_fa, description_en, description_fa)
        self._execute_and_commit(sql, values)
        print("Saved in images successfully...")
        
    def save_pdfs(self, parent_id, parent_type, pdf_URL, description_en, description_fa):
        sql = """INSERT INTO `pdfs`
                 (`parent_UID`, `parent_type`, `pdf_URL`, `description_en`, `description_fa`) 
                 VALUES (%s,%s,%s,%s,%s)"""
        values = (parent_id, parent_type, pdf_URL, description_en, description_fa)
        self._execute_and_commit(sql, values)
        print("Saved in pdfs successfully...")
    
    
    def save_videos(self, brand_en, brand_fa, parent_id, parent_type, video_URL, 
                  alt_en, alt_fa, description_en, description_fa):
        sql = """INSERT INTO `pdfs`
                 (`brand_en`, `brand_fa`,`parent_UID`, `parent_type`, `video_URL`, 
                 `alt_en`, `alt_fa`, `description_en`, `description_fa`) 
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        values = (brand_en, brand_fa, parent_id, parent_type, pdf_URL, 
                  alt_en, alt_fa, description_en, description_fa)
        self._execute_and_commit(sql, values)
        print("Saved in pdfs successfully...")